# Mini Project 1 (Part 3)

## Expand the scope to look at the data outside of just the Philadelphia School District.  Using plot.ly, plot results



In [228]:
import pandas as pd
school_data = pd.read_csv('userssharedsdfschoolimprovement2010grants.csv')
school_data.columns = school_data.columns.str.replace(' ', '_')
school_data.rename(columns = {'2010/11/Award_Amount':'Amount'}, inplace=True)
school_data['Amount'] = school_data['Amount'].str.replace('$','')
school_data['Amount'] = school_data.Amount.astype(float)
school_data['School_Name'] = school_data.School_Name.str.upper()
school_data['City'] = school_data.City.str.upper()
school_data['State'] = school_data.State.str.upper()
school_data['District_Name'] = school_data.District_Name.str.upper()
school_data['Model_Selected'] = school_data['Model_Selected'].str.upper()
school_data.head()

,School_Name,City,State,District_Name,Amount,Model_Selected,Location
0,HOGARTH KINGEEKUK MEMORIAL SCHOOL,SAVOONGA,AK,BERING STRAIT SCHOOL DISTRICT,471014.0,TRANSFORMATION,"200 MAIN ST\nSAVOONGA, AK 99769\n(63.6687, -17..."
1,AKIACHAK SCHOOL,AKIACHAK,AK,YUPIIT SCHOOL DISTRICT,520579.0,TRANSFORMATION,"AKIACHAK 51100\nAKIACHAK, AK 99551\n(60.8911, ..."
2,GAMBELL SCHOOL,GAMBELL,AK,BERING STRAIT SCHOOL DISTRICT,449592.0,TRANSFORMATION,"169 MAIN ST\nGAMBELL, AK 99742\n(63.7413, -171..."
3,BURCHELL HIGH SCHOOL,WASILLA,AK,MATANUSKA-SUSITNA BOROUGH SCHOOL DISTRICT,641184.0,TRANSFORMATION,"1775 WEST PARKS HWY\nWASILLA, AK 99654\n(61.57..."
4,AKIAK SCHOOL,AKIAK,AK,YUPIIT SCHOOL DISTRICT,399686.0,TRANSFORMATION,"AKIAK 5227\nAKIAK, AK 99552\n(60.8879, -161.2)"


# Add the GPS data a a separate column.

In [229]:
gps = school_data.Location.str.split("\n").apply(pd.Series)[2].str.replace(",", "").str.replace(")","").str.replace("(","").str.split(",").apply(pd.Series)
gps
school_data['GPS'] = gps
school_data.head()

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
0,HOGARTH KINGEEKUK MEMORIAL SCHOOL,SAVOONGA,AK,BERING STRAIT SCHOOL DISTRICT,471014.0,TRANSFORMATION,"200 MAIN ST\nSAVOONGA, AK 99769\n(63.6687, -17...",63.6687 -170.603
1,AKIACHAK SCHOOL,AKIACHAK,AK,YUPIIT SCHOOL DISTRICT,520579.0,TRANSFORMATION,"AKIACHAK 51100\nAKIACHAK, AK 99551\n(60.8911, ...",60.8911 -161.376
2,GAMBELL SCHOOL,GAMBELL,AK,BERING STRAIT SCHOOL DISTRICT,449592.0,TRANSFORMATION,"169 MAIN ST\nGAMBELL, AK 99742\n(63.7413, -171...",63.7413 -171.689
3,BURCHELL HIGH SCHOOL,WASILLA,AK,MATANUSKA-SUSITNA BOROUGH SCHOOL DISTRICT,641184.0,TRANSFORMATION,"1775 WEST PARKS HWY\nWASILLA, AK 99654\n(61.57...",61.5794 -149.495
4,AKIAK SCHOOL,AKIAK,AK,YUPIIT SCHOOL DISTRICT,399686.0,TRANSFORMATION,"AKIAK 5227\nAKIAK, AK 99552\n(60.8879, -161.2)",60.8879 -161.2


# Expect to see almost $3,500,000,000 if this data is all of the awards; result is not as expected.  This file appears to be a sample or missing a lot of data.  Only 757 of the 831 records have an amount filled in.  That is another problem with the data.  Let's look first at the uniqueness of the records.

In [230]:
school_data.sum(axis=0,numeric_only=True)  

Amount    318566716.0
dtype: float64

In [231]:
school_data.describe()

,Amount
count,757.000000
mean,420827.894320
std,287458.053183
min,1.000000
25%,162541.000000
50%,407794.000000
75%,671142.000000
max,997852.000000


In [232]:
school_data.describe(include=['object'])

,School_Name,City,State,District_Name,Model_Selected,Location,GPS
count,831,831,831,831,825,831,831
unique,821,422,50,419,4,823,821
top,RIO VISTA ELEMENTARY,PHILADELPHIA,CA,PHILADELPHIA CITY SD,TRANSFORMATION,"6941 NORTH CENTRAL ST\nPORTLAND, OR 97203\n(45...",45.5899 -122.739
freq,2,29,92,27,608,3,3


# From looking at the above, I can see there are 821 unique school names (so 10 of the records are a repeat of another School_Name); there are 422 unique City; all 50 States are represented; there are 419 different Districts; 4 types of Models, 823 unique locations and 821 unique GPS locations. 

School_Name - Rio Vista Elementary is the Top School with a Frequency of 2.  This leads me to believe there are other schools with 2 as occurrences as there are 10 non-unique schools.  If the top only has 2 occurrences, there should be 10 schools with 2 occurrences to account for the non-unique 10 School_Name Values.  To look for and verify this, below sorts the data, looks for duplicates and lists the duplicates.  It does return 10 duplicate schools.  

City - some cities received more than 1 grant as there are only 422 unique cities

State - wouldn't expect to see more than 51 because DC is counted in with State.  So this looks OK.

District_Name - There are 419.  As there were only 422 cities, this looks OK.  Cities would normally be one school district.  In some cases, a school district can cover 2 cities/towns.

Model_Selected - 4 types of models is as expected since there were 4 different grant programs; however, 6 records do not specify which Model they selected when receiving a grant.  This is missing data.

Location - There are 823 unique.  Eight locations have the address repeated - so this could be multiple grants to one location.

GPS - There are 821 unique addresses.  I would have expected to see the same number of GPS as Locations - unless it is a case issue or other input issue with location as I didn't change the case of the location.

In [233]:
school_data.sort_values('School_Name', inplace = True)
bool_series = school_data.duplicated('School_Name')
dups = school_data[bool_series]
dups

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
91,BURBANK ELEMENTARY,HAYWARD,CA,HAYWARD UNIFIED,158712.0,TRANSFORMATION,"353 B ST\nHAYWARD, CA 94541\n(37.6679, -122.094)",37.6679 -122.094
762,CENTRAL HIGH,KING AND QUEEN C H,VA,KING AND QUEEN CO PBLC SCHS,685661.0,TRANSFORMATION,"17024 THE TRAIL\nKING AND QUEEN C H, VA 23085\...",37.6753 -76.8836
198,CENTRAL HIGH SCHOOL,BROOKSVILLE,FL,HERNANDO,737052.0,TRANSFORMATION,"14075 KEN AUSTIN PKWY\nBROOKSVILLE, FL 34613\n...",28.5647 -82.4956
540,EAST HIGH SCHOOL,ROCHESTER,NY,ROCHESTER CITY SCHOOL DISTRICT,19525.0,TRANSFORMATION,"1801 EAST MAIN ST\nROCHESTER, NY 14609\n(43.15...",43.1581 -77.5649
704,FRANKLIN D ROOSEVELT HIGH SCHOOL,DALLAS,TX,DALLAS ISD,NaN,TRANSFORMATION,"525 BONNIE VIEW RD\nDALLAS, TX 75203\n(32.7423...",32.7423 -96.7966
180,MOUNT PLEASANT HIGH SCHOOL,WILMINGTON,DE,BRANDYWINE SCHOOL DISTRICT,672921.0,TRANSFORMATION,"5201 WASHINGTON ST EXTENSI\nWILMINGTON, DE 198...",39.7724 -75.5041
136,NORTH HIGH SCHOOL,DENVER,CO,SCHOOL DISTRICT NO. 1 IN THE COUNTY OF DENVER ...,924081.0,TRANSFORMATION,"2960 NORTH SPEER BLVD\nDENVER, CO 80211\n(39.7...",39.7602 -105.02
752,NORTHWEST MIDDLE,SALT LAKE CITY,UT,SALT LAKE DISTRICT,806790.0,TRANSFORMATION,"1730 WEST 1700 NORTH\nSALT LAKE CITY, UT 84116...",40.8057 -111.94
128,RIO VISTA ELEMENTARY,BAY POINT,CA,MT. DIABLO UNIFIED,443230.0,TRANSFORMATION,"611 PACIFICA AVE\nBAY POINT, CA 94565\n(38.033...",38.0333 -121.975
524,ROOSEVELT HIGH SCHOOL,YONKERS,NY,YONKERS CITY SCHOOL DISTRICT,2.0,TRANSFORMATION,"631 TUCKAHOE RD\nYONKERS, NY 10710\n(40.9538, ...",40.9538 -73.845


# The other, easier way to do it is to count the number of times a value shows up.  

In [234]:
school_data['School_Name'].value_counts()

ROOSEVELT HIGH SCHOOL                                 2
CENTRAL HIGH SCHOOL                                   2
NORTH HIGH SCHOOL                                     2
FRANKLIN D ROOSEVELT HIGH SCHOOL                      2
RIO VISTA ELEMENTARY                                  2
CENTRAL HIGH                                          2
BURBANK ELEMENTARY                                    2
EAST HIGH SCHOOL                                      2
NORTHWEST MIDDLE                                      2
MOUNT PLEASANT HIGH SCHOOL                            2
BOOKER T. WASHINGTON SENIOR HIGH                      1
LYBROOK ELEMENTARY                                    1
NORTHSIDE HIGH SCHOOL                                 1
LAKESIDE ELEMENTARY                                   1
GLENDALE MIDDLE                                       1
THORNRIDGE HIGH SCHOOL                                1
GEORGE L DILWORTH MIDDLE SCHOOL                       1
DR. HENRY W. MACK/WEST LITTLE RIVER ELEMENTARY S

# Are these truly duplicate school records or records for another school that has the same name?   

They are different schools. We do have 831 unique records.  


In [235]:
school_data[(school_data.School_Name == 'EAST HIGH SCHOOL') | (school_data.School_Name =='ROOSEVELT HIGH SCHOOL') | (school_data.School_Name == 'FRANKLIN D ROOSEVELT HIGH SCHOOL') | 
            (school_data.School_Name == 'MOUNT PLEASANT HIGH SCHOOL')  | (school_data.School_Name == 'RIO VISTA ELEMENTARY') | (school_data.School_Name == 'CENTRAL HIGH') | 
            (school_data.School_Name == 'CENTRAL HIGH SCHOOL') | (school_data.School_Name == 'NORTH HIGH SCHOOL') | (school_data.School_Name == 'NORTHWEST MIDDLE') | 
            (school_data.School_Name == 'BURBANK ELEMENTARY')]

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
88,BURBANK ELEMENTARY,SAN DIEGO,CA,SAN DIEGO UNIFIED,116423.0,TRANSFORMATION,"2146 JULIAN AVE\nSAN DIEGO, CA 92113\n(32.7009...",32.7009 -117.138
91,BURBANK ELEMENTARY,HAYWARD,CA,HAYWARD UNIFIED,158712.0,TRANSFORMATION,"353 B ST\nHAYWARD, CA 94541\n(37.6679, -122.094)",37.6679 -122.094
403,CENTRAL HIGH,KANSAS CITY,MO,KANSAS CITY 33,806900.0,TRANSFORMATION,"3221 INDIANA\nKANSAS CITY, MO 64128\n(39.0669,...",39.0669 -94.5435
762,CENTRAL HIGH,KING AND QUEEN C H,VA,KING AND QUEEN CO PBLC SCHS,685661.0,TRANSFORMATION,"17024 THE TRAIL\nKING AND QUEEN C H, VA 23085\...",37.6753 -76.8836
139,CENTRAL HIGH SCHOOL,PUEBLO,CO,"PUEBLO, SCHOOL DISTRICT NO. 60, IN THE COUNTY ...",852911.0,TRANSFORMATION,"216 EAST ORMAN AVE\nPUEBLO, CO 81004\n(38.2536...",38.2536 -104.626
198,CENTRAL HIGH SCHOOL,BROOKSVILLE,FL,HERNANDO,737052.0,TRANSFORMATION,"14075 KEN AUSTIN PKWY\nBROOKSVILLE, FL 34613\n...",28.5647 -82.4956
548,EAST HIGH SCHOOL,YOUNGSTOWN,OH,YOUNGSTOWN CITY SCHOOLS,805793.0,TRANSFORMATION,"474 BENNINGTON AVE\nYOUNGSTOWN, OH 44505\n(41....",41.1041 -80.6196
540,EAST HIGH SCHOOL,ROCHESTER,NY,ROCHESTER CITY SCHOOL DISTRICT,19525.0,TRANSFORMATION,"1801 EAST MAIN ST\nROCHESTER, NY 14609\n(43.15...",43.1581 -77.5649
529,FRANKLIN D ROOSEVELT HIGH SCHOOL,BROOKLYN,NY,NEW YORK CITY GEOGRAPHIC DISTRICT #20,18.0,TRANSFORMATION,"5800 20TH AVE\nBROOKLYN, NY 11204\n(40.6205, -...",40.6205 -73.9817
704,FRANKLIN D ROOSEVELT HIGH SCHOOL,DALLAS,TX,DALLAS ISD,NaN,TRANSFORMATION,"525 BONNIE VIEW RD\nDALLAS, TX 75203\n(32.7423...",32.7423 -96.7966


# Look at the records with no Model_Selected info to see if they have a grant amount.  

Expect to see 6 records. Results show that.  All of these have no grant so the statistics computed on grants by Model_Selected will not be affected by these records.  It is interesting that these are all schools in Rhode Island.  Was there a problem with submitting data in Rhode Island?  Checked that next to see if there was any data for Rhode Island other than these 6 records.  There is not, so Rhode Island data is not included in the data



In [236]:
school_data[(school_data.Model_Selected != 'TRANSFORMATION') & (school_data.Model_Selected != 'TURNAROUND') & (school_data.Model_Selected != 'RESTART') & (school_data.Model_Selected != 'CLOSURE')]

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
666,CENTRAL FALLS SENIOR HIGH SCHOOL,CENTRAL FALLS,RI,CENTRAL FALLS,NaN,NaN,"24 SUMMER ST\nCENTRAL FALLS, RI 2863\n(41.8872...",41.8872 -71.3916
661,CHARLOTTE WOODS ELEMENTARY SCHOOL,PROVIDENCE,RI,PROVIDENCE,NaN,NaN,"674 PRAIRIE AVE\nPROVIDENCE, RI 2905\n(41.7957...",41.7957 -71.4106
664,FEINSTEIN HIGH SCHOOL,PROVIDENCE,RI,PROVIDENCE,NaN,NaN,"544 ELMWOOD AVE\nPROVIDENCE, RI 2907\n(41.7977...",41.7977 -71.4254
662,"LILLIAN FEINSTEIN ELEMENTARY, SACKETT STREET",PROVIDENCE,RI,PROVIDENCE,NaN,NaN,"159 SACKETT ST\nPROVIDENCE, RI 2907\n(41.794, ...",41.794 -71.4193
663,ROGER WILLIAMS MIDDLE SCHOOL,PROVIDENCE,RI,PROVIDENCE,NaN,NaN,"278 THURBERS AVE\nPROVIDENCE, RI 2905\n(41.798...",41.7984 -71.4109
665,WILLIAM B. COOLEY/HEALTH AND SCIENCE TECH. ACA...,PROVIDENCE,RI,PROVIDENCE,NaN,NaN,"182 THURBERS AVE\nPROVIDENCE, RI 2905\n(41.798...",41.7981 -71.4078


In [237]:
school_data[school_data.State == 'RI']

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
666,CENTRAL FALLS SENIOR HIGH SCHOOL,CENTRAL FALLS,RI,CENTRAL FALLS,NaN,NaN,"24 SUMMER ST\nCENTRAL FALLS, RI 2863\n(41.8872...",41.8872 -71.3916
661,CHARLOTTE WOODS ELEMENTARY SCHOOL,PROVIDENCE,RI,PROVIDENCE,NaN,NaN,"674 PRAIRIE AVE\nPROVIDENCE, RI 2905\n(41.7957...",41.7957 -71.4106
664,FEINSTEIN HIGH SCHOOL,PROVIDENCE,RI,PROVIDENCE,NaN,NaN,"544 ELMWOOD AVE\nPROVIDENCE, RI 2907\n(41.7977...",41.7977 -71.4254
662,"LILLIAN FEINSTEIN ELEMENTARY, SACKETT STREET",PROVIDENCE,RI,PROVIDENCE,NaN,NaN,"159 SACKETT ST\nPROVIDENCE, RI 2907\n(41.794, ...",41.794 -71.4193
663,ROGER WILLIAMS MIDDLE SCHOOL,PROVIDENCE,RI,PROVIDENCE,NaN,NaN,"278 THURBERS AVE\nPROVIDENCE, RI 2905\n(41.798...",41.7984 -71.4109
665,WILLIAM B. COOLEY/HEALTH AND SCIENCE TECH. ACA...,PROVIDENCE,RI,PROVIDENCE,NaN,NaN,"182 THURBERS AVE\nPROVIDENCE, RI 2905\n(41.798...",41.7981 -71.4078


# Looking at duplicate Locations to see if they received multiple grants or if multiple schools are located at the same address.  

The school names are different so it appears they are different schools at the same location.  Expected 8 records.  This was only 15, not 16 schools as one address is associated with 3, not 2, schools which is in line with the top frequency of 3.


In [238]:
school_data.sort_values('Location', inplace = True)
bool_series = school_data.duplicated('Location')
dups = school_data[bool_series]
dups


,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
442,PRYOR 7-8,PRYOR,MT,PRYOR ELEM,134368.0,TRANSFORMATION,"1 HIGH SCHOOL LN\nPRYOR, MT 59066\n(45.4297, -...",45.4297 -108.533
477,SANTEE HIGH SCHOOL,NIOBRARA,NE,SANTEE COMMUNITY SCHOOLS,542500.0,TRANSFORMATION,"206 FRAZIER AVE EAST\nNIOBRARA, NE 68760\n(42....",42.8394 -97.8506
482,NUTE JUNIOR HIGH SCHOOL,MILTON,NH,MILTON SCHOOL DISTRICT,337678.0,TRANSFORMATION,"22 ELM ST\nMILTON, NH 3851\n(43.4084, -70.9878)",43.4084 -70.9878
484,PITTSFIELD MIDDLE SCHOOL,PITTSFIELD,NH,PITTSFIELD SCHOOL DISTRICT,344358.0,TRANSFORMATION,"23 ONEIDA ST\nPITTSFIELD, NH 3263\n(43.3029, -...",43.3029 -71.3181
395,HMONG INTERNATIONAL ACADEMY,MINNEAPOLIS,MN,MINNEAPOLIS PUBLIC SCHOOL DIST.,528276.0,TRANSFORMATION,"3333 PENN AVE NORTH\nMINNEAPOLIS, MN 55412\n(4...",45.0156 -93.3083
525,ROBERT BROWN HIGH SCHOOL OF CONSTRUCTION AND D...,ROCHESTER,NY,ROCHESTER CITY SCHOOL DISTRICT,178685.0,TURNAROUND,"655 Colfax St\nRochester, NY 14606\n(43.1779, ...",43.1779 -77.6724
596,"ARTS, COMMUNICATION & TECHNOLOGY SCHOOL",PORTLAND,OR,PORTLAND SD 1J,766667.0,TRANSFORMATION,"6941 NORTH CENTRAL ST\nPORTLAND, OR 97203\n(45...",45.5899 -122.739
599,PURSUIT OF WELLNESS EDUCATION AT ROOSEVELT CAMPUS,PORTLAND,OR,PORTLAND SD 1J,119072.0,TRANSFORMATION,"6941 NORTH CENTRAL ST\nPORTLAND, OR 97203\n(45...",45.5899 -122.739


In [260]:
school_data[(school_data.Location.str.contains('1 HIGH SCHOOL LN')) |
          (school_data.Location.str.contains('206 FRAZIER AVE EAST')) | 
          (school_data.Location.str.contains('22 ELM ST')) | 
          (school_data.Location.str.contains('23 ONEIDA ST')) | 
          (school_data.Location.str.contains('3333 PENN AVE NORTH')) | 
          (school_data.Location.str.contains('655 Colfax St')) | 
          (school_data.Location.str.contains('6941 NORTH CENTRAL ST'))]

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
441,PLENTY COUPS HIGH SCHOOL,PRYOR,MT,PLENTY COUPS H S,380711.0,TRANSFORMATION,"1 HIGH SCHOOL LN\nPRYOR, MT 59066\n(45.4297, -...",45.4297 -108.533
442,PRYOR 7-8,PRYOR,MT,PRYOR ELEM,134368.0,TRANSFORMATION,"1 HIGH SCHOOL LN\nPRYOR, MT 59066\n(45.4297, -...",45.4297 -108.533
472,SANTEE ELEMENTARY SCHOOL,NIOBRARA,NE,SANTEE COMMUNITY SCHOOLS,510953.0,TRANSFORMATION,"206 FRAZIER AVE EAST\nNIOBRARA, NE 68760\n(42....",42.8394 -97.8506
477,SANTEE HIGH SCHOOL,NIOBRARA,NE,SANTEE COMMUNITY SCHOOLS,542500.0,TRANSFORMATION,"206 FRAZIER AVE EAST\nNIOBRARA, NE 68760\n(42....",42.8394 -97.8506
480,NUTE HIGH SCHOOL,MILTON,NH,MILTON SCHOOL DISTRICT,421278.0,TRANSFORMATION,"22 ELM ST\nMILTON, NH 3851\n(43.4084, -70.9878)",43.4084 -70.9878
482,NUTE JUNIOR HIGH SCHOOL,MILTON,NH,MILTON SCHOOL DISTRICT,337678.0,TRANSFORMATION,"22 ELM ST\nMILTON, NH 3851\n(43.4084, -70.9878)",43.4084 -70.9878
479,PITTSFIELD HIGH SCHOOL,PITTSFIELD,NH,PITTSFIELD SCHOOL DISTRICT,344358.0,TRANSFORMATION,"23 ONEIDA ST\nPITTSFIELD, NH 3263\n(43.3029, -...",43.3029 -71.3181
484,PITTSFIELD MIDDLE SCHOOL,PITTSFIELD,NH,PITTSFIELD SCHOOL DISTRICT,344358.0,TRANSFORMATION,"23 ONEIDA ST\nPITTSFIELD, NH 3263\n(43.3029, -...",43.3029 -71.3181
396,LUCY LANEY @ CLEVELAND PARK ELEM.,MINNEAPOLIS,MN,MINNEAPOLIS PUBLIC SCHOOL DIST.,728276.0,TURNAROUND,"3333 PENN AVE NORTH\nMINNEAPOLIS, MN 55412\n(4...",45.0156 -93.3083
395,HMONG INTERNATIONAL ACADEMY,MINNEAPOLIS,MN,MINNEAPOLIS PUBLIC SCHOOL DIST.,528276.0,TRANSFORMATION,"3333 PENN AVE NORTH\nMINNEAPOLIS, MN 55412\n(4...",45.0156 -93.3083


# GPS records

Duplicate locations are different schools at the same address.  But why is the unique GPS count not the same as the unique Locations count if each address has a unique GPS location?  I would have expected them to be the same, but there is a difference of 10, not 8.  The top frequency is 3 so I expect 19 or fewer records returned.  I have 19 records.  One GPS address is associated with 2 physical addresses that are different.  One GPS is address is associated with 1 physical address that is in 2 different cases.  So one GPS is probably incorrect as the house numbers are 801 North Coyote and 603 North Coyote.  The other GPS would be a correct duplicate address, but different school. 


In [262]:
school_data.sort_values('GPS', inplace = True)
bool_series = school_data.duplicated('GPS')
dups = school_data[bool_series]
dups

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
741,LA JOYA PALMVIEW H S,LA JOYA,TX,LA JOYA ISD,NaN,TRANSFORMATION,"603 NORTH COYOTE DR\nLA JOYA, TX 78560\n(26.23...",26.2358 -98.4771
472,SANTEE ELEMENTARY SCHOOL,NIOBRARA,NE,SANTEE COMMUNITY SCHOOLS,510953.0,TRANSFORMATION,"206 FRAZIER AVE EAST\nNIOBRARA, NE 68760\n(42....",42.8394 -97.8506
543,ROCHESTER STEM HIGH SCHOOL,ROCHESTER,NY,ROCHESTER CITY SCHOOL DISTRICT,185579.0,TURNAROUND,"655 Colfax St\nRochester, NY 14606\n(43.1779, ...",43.1779 -77.6724
534,GLOBAL MEDIA ARTS HS AT FRANKLIN,ROCHESTER,NY,ROCHESTER CITY SCHOOL DISTRICT,137761.0,TURNAROUND,"950 NORTON ST\nROCHESTER, NY 14621\n(43.1886, ...",43.1886 -77.5981
479,PITTSFIELD HIGH SCHOOL,PITTSFIELD,NH,PITTSFIELD SCHOOL DISTRICT,344358.0,TRANSFORMATION,"23 ONEIDA ST\nPITTSFIELD, NH 3263\n(43.3029, -...",43.3029 -71.3181
482,NUTE JUNIOR HIGH SCHOOL,MILTON,NH,MILTON SCHOOL DISTRICT,337678.0,TRANSFORMATION,"22 ELM ST\nMILTON, NH 3851\n(43.4084, -70.9878)",43.4084 -70.9878
395,HMONG INTERNATIONAL ACADEMY,MINNEAPOLIS,MN,MINNEAPOLIS PUBLIC SCHOOL DIST.,528276.0,TRANSFORMATION,"3333 PENN AVE NORTH\nMINNEAPOLIS, MN 55412\n(4...",45.0156 -93.3083
442,PRYOR 7-8,PRYOR,MT,PRYOR ELEM,134368.0,TRANSFORMATION,"1 HIGH SCHOOL LN\nPRYOR, MT 59066\n(45.4297, -...",45.4297 -108.533
596,"ARTS, COMMUNICATION & TECHNOLOGY SCHOOL",PORTLAND,OR,PORTLAND SD 1J,766667.0,TRANSFORMATION,"6941 NORTH CENTRAL ST\nPORTLAND, OR 97203\n(45...",45.5899 -122.739
599,PURSUIT OF WELLNESS EDUCATION AT ROOSEVELT CAMPUS,PORTLAND,OR,PORTLAND SD 1J,119072.0,TRANSFORMATION,"6941 NORTH CENTRAL ST\nPORTLAND, OR 97203\n(45...",45.5899 -122.739


In [264]:
school_data[(school_data.GPS == '26.2358 -98.4771') | (school_data.GPS =='42.8394 -97.8506') | (school_data.GPS == '43.1779 -77.6724') | 
            (school_data.GPS == '43.1886 -77.5981')  | (school_data.GPS == '43.3029 -71.3181') | (school_data.GPS == '43.4084 -70.9878') | 
            (school_data.GPS == '45.0156 -93.3083') | (school_data.GPS == '45.4297 -108.533') | (school_data.GPS == '45.5899 -122.739')]
           

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
705,JUAREZ-LINCOLN H S,LA JOYA,TX,LA JOYA ISD,NaN,TRANSFORMATION,"801 NORTH COYOTE DR\nLA JOYA, TX 78560\n(26.23...",26.2358 -98.4771
741,LA JOYA PALMVIEW H S,LA JOYA,TX,LA JOYA ISD,NaN,TRANSFORMATION,"603 NORTH COYOTE DR\nLA JOYA, TX 78560\n(26.23...",26.2358 -98.4771
477,SANTEE HIGH SCHOOL,NIOBRARA,NE,SANTEE COMMUNITY SCHOOLS,542500.0,TRANSFORMATION,"206 FRAZIER AVE EAST\nNIOBRARA, NE 68760\n(42....",42.8394 -97.8506
472,SANTEE ELEMENTARY SCHOOL,NIOBRARA,NE,SANTEE COMMUNITY SCHOOLS,510953.0,TRANSFORMATION,"206 FRAZIER AVE EAST\nNIOBRARA, NE 68760\n(42....",42.8394 -97.8506
525,ROBERT BROWN HIGH SCHOOL OF CONSTRUCTION AND D...,ROCHESTER,NY,ROCHESTER CITY SCHOOL DISTRICT,178685.0,TURNAROUND,"655 Colfax St\nRochester, NY 14606\n(43.1779, ...",43.1779 -77.6724
543,ROCHESTER STEM HIGH SCHOOL,ROCHESTER,NY,ROCHESTER CITY SCHOOL DISTRICT,185579.0,TURNAROUND,"655 Colfax St\nRochester, NY 14606\n(43.1779, ...",43.1779 -77.6724
532,VANGUARD COLLEGIATE HIGH SCHOOL,ROCHESTER,NY,ROCHESTER CITY SCHOOL DISTRICT,122114.0,TURNAROUND,"950 Norton St\nRochester, NY 14621\n(43.1886, ...",43.1886 -77.5981
534,GLOBAL MEDIA ARTS HS AT FRANKLIN,ROCHESTER,NY,ROCHESTER CITY SCHOOL DISTRICT,137761.0,TURNAROUND,"950 NORTON ST\nROCHESTER, NY 14621\n(43.1886, ...",43.1886 -77.5981
484,PITTSFIELD MIDDLE SCHOOL,PITTSFIELD,NH,PITTSFIELD SCHOOL DISTRICT,344358.0,TRANSFORMATION,"23 ONEIDA ST\nPITTSFIELD, NH 3263\n(43.3029, -...",43.3029 -71.3181
479,PITTSFIELD HIGH SCHOOL,PITTSFIELD,NH,PITTSFIELD SCHOOL DISTRICT,344358.0,TRANSFORMATION,"23 ONEIDA ST\nPITTSFIELD, NH 3263\n(43.3029, -...",43.3029 -71.3181


# What records are missing the grant data and is there anything in common with them?  

First I verify that I have the 74 records.  Then I group by state and see if these records are the only records for those states.

In [276]:
bool_series = school_data.Amount >= 0
missing_amount = school_data[~bool_series]
missing_amount


,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
708,SENTRY TECHNOLOGY PREP SCH,BROWNSVILLE,TX,ONE STOP MULTISERVICE CHARTER SCHOOL,NaN,TRANSFORMATION,"508 EAST ELIZABETH\nBROWNSVILLE, TX 78520\n(25...",25.9057 -97.5031
715,MERCEDES ACADEMIC ACADEMY,MERCEDES,TX,MERCEDES ISD,NaN,TRANSFORMATION,"333 SOUTH OHIO ST\nMERCEDES, TX 78570\n(26.148...",26.1486 -97.9122
712,MID-VALLEY ACADEMY,MERCEDES,TX,MID-VALLEY ACADEMY,NaN,TRANSFORMATION,"103 EAST 2ND ST\nMERCEDES, TX 78570\n(26.15, -...",26.15 -97.9095
720,ONE STOP MULTISERVICE,WESLACO,TX,ONE STOP MULTISERVICE CHARTER SCHOOL,NaN,TRANSFORMATION,"615 SOUTH INTERNATIONAL\nWESLACO, TX 78596\n(2...",26.1538 -97.9584
735,MID-VALLEY ACADEMY (9-12),SAN BENITO,TX,MID-VALLEY ACADEMY,NaN,TRANSFORMATION,"1785 WEST BUSINESS HWY 77\nSAN BENITO, TX 7858...",26.1558 -97.6512
748,MID-VALLEY ACADEMY-MCALLEN,MCALLEN,TX,MID-VALLEY ACADEMY,NaN,TRANSFORMATION,"200 NORTH 17TH ST\nMCALLEN, TX 78501\n(26.206,...",26.206 -98.2375
705,JUAREZ-LINCOLN H S,LA JOYA,TX,LA JOYA ISD,NaN,TRANSFORMATION,"801 NORTH COYOTE DR\nLA JOYA, TX 78560\n(26.23...",26.2358 -98.4771
741,LA JOYA PALMVIEW H S,LA JOYA,TX,LA JOYA ISD,NaN,TRANSFORMATION,"603 NORTH COYOTE DR\nLA JOYA, TX 78560\n(26.23...",26.2358 -98.4771
726,JUVENILE DETENTION CTR,EDINBURG,TX,EDINBURG CISD,NaN,TRANSFORMATION,"3100 SOUTH HWY 281\nEDINBURG, TX 78539\n(26.27...",26.2749 -98.1703
743,CHILDREN OF THE SUN,RIO GRANDE CITY,TX,ONE STOP MULTISERVICE CHARTER SCHOOL,NaN,TRANSFORMATION,"5324 EAST US HWY 83 STE 2\nRIO GRANDE CITY, TX...",26.3549 -98.7723


In [277]:
missing_amount = school_data[~bool_series].groupby('State').count()
missing_amount

,School_Name,City,District_Name,Amount,Model_Selected,Location,GPS
State,,,,,,,
IA,2,2,2,0,2,2,2
ID,4,4,4,0,4,4,4
LA,2,2,2,0,2,2,2
ME,6,6,6,0,6,6,6
NC,1,1,1,0,1,1,1
NM,4,4,4,0,4,4,4
RI,6,6,6,0,0,6,6
SD,1,1,1,0,1,1,1
TX,48,48,48,0,48,48,48


# State reporting problems?

There are 9 states that have the missing grant amounts.  I already know that all of RI's records are missing the grant data amount.  Is that the same case for the other 8 states with missing values?  From looking below, it appears that Texas, Maine, and Louisiana also have problems with reporting any data and are also not represented in the supplied data.  2/3 of Iowa, 1/3 of Idaho and South Dakota, 1 of 24 in North Carolina and 4 of 13 in New Mexico have a missing grant value.  So although 50 states appear in the data, there is only grant amount information included for 46 of the 50.  Five of the remaining 46 states have missing data in their reports.  Since this is a sample and not the entire list of grant recipients, I will use the amounts provided, but drop off the locations where there is no amount listed.

In [278]:
school_data[school_data.State == 'IA']

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
278,WEEKS MIDDLE SCHOOL,DES MOINES,IA,DES MOINES INDEPENDENT COMM SCHOOL DISTRICT,18.0,TRANSFORMATION,"901 EAST PARK AVE\nDES MOINES, IA 50315\n(41.5...",41.554 -93.6005
283,EDMUNDS FINE ARTS ACADEMY,DES MOINES,IA,DES MOINES INDEPENDENT COMM SCHOOL DISTRICT,NaN,TRANSFORMATION,"1601 CROCKER ST\nDES MOINES, IA 50314\n(41.594...",41.5943 -93.639
279,NORTH HIGH SCHOOL,DES MOINES,IA,DES MOINES INDEPENDENT COMM SCHOOL DISTRICT,2.0,TRANSFORMATION,"501 HOLCOMB AVE\nDES MOINES, IA 50313\n(41.617...",41.6179 -93.6219
281,HOYT MIDDLE SCHOOL,DES MOINES,IA,DES MOINES INDEPENDENT COMM SCHOOL DISTRICT,18.0,TRANSFORMATION,"2700 EAST 42ND ST\nDES MOINES, IA 50317\n(41.6...",41.6206 -93.5335
280,LINCOLN ELEMENTARY SCHOOL,WATERLOO,IA,WATERLOO COMM SCHOOL DISTRICT,NaN,TRANSFORMATION,"302 CEDAR BEND\nWATERLOO, IA 50703\n(42.5186, ...",42.5186 -92.3647
282,JACK M LOGAN MIDDLE SCHOOL,WATERLOO,IA,WATERLOO COMM SCHOOL DISTRICT,761197.0,TRANSFORMATION,"1515 LOGAN AVE\nWATERLOO, IA 50703\n(42.5215, ...",42.5215 -92.3378


In [279]:
school_data[school_data.State == 'ID']

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
284,MURTAUGH MIDDLE SCHOOL,MURTAUGH,ID,MURTAUGH JOINT DISTRICT,209893.0,TRANSFORMATION,"500 WEST BOYD\nMURTAUGH, ID 83344\n(42.4923, -...",42.4923 -114.159
285,FORT HALL ELEMENTARY SCHOOL,POCATELLO,ID,BLACKFOOT DISTRICT,NaN,TRANSFORMATION,"B ST\nPOCATELLO, ID 83202\n(42.906, -112.582)",42.906 -112.582
288,SNAKE RIVER JR HIGH SCHOOL,BLACKFOOT,ID,SNAKE RIVER DISTRICT,NaN,TRANSFORMATION,"918 WEST HWY 39\nBLACKFOOT, ID 83221\n(43.197,...",43.197 -112.478
286,MELBA MIDDLE SCHOOL,MELBA,ID,MELBA JOINT DISTRICT,NaN,TRANSFORMATION,"560 BRDWAY\nMELBA, ID 83641\n(43.3785, -116.531)",43.3785 -116.531
289,JEFFERSON MIDDLE SCHOOL,CALDWELL,ID,CALDWELL DISTRICT,286787.0,TRANSFORMATION,"3311 SOUTH 10TH AVE\nCALDWELL, ID 83605\n(43.6...",43.6407 -116.693
287,LAKESIDE ELEMENTARY SCHOOL,WORLEY,ID,PLUMMER-WORLEY JOINT DISTRICT,NaN,TRANSFORMATION,"S 29900 FIRST AVE\nWORLEY, ID 83876\n(47.4, -1...",47.4 -116.916


In [280]:
school_data[school_data.State == 'LA']

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
324,ASSUMPTION HIGH SCHOOL,NAPOLEONVILLE,LA,ASSUMPTION PARISH,NaN,TRANSFORMATION,"4880 HWY 308\nNAPOLEONVILLE, LA 70390\n(29.941...",29.9416 -91.0224
323,SARAH TOWLES REED SENIOR HIGH SCHOOL,NEW ORLEANS,LA,RECOVERY SCHOOL DISTRICT-LDE,NaN,TRANSFORMATION,"5316 MICHOUD BLVD\nNEW ORLEANS, LA 70129\n(30....",30.0405 -89.9304


In [281]:
school_data[school_data.State == 'ME']

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
352,RIVERTON SCHOOL,PORTLAND,ME,PORTLAND PUBLIC SCHOOLS,NaN,TRANSFORMATION,"1600 FOREST AVE\nPORTLAND, ME 4101\n(43.6972, ...",43.6972 -70.3075
349,LAKE REGION HIGH SCHOOL,NAPLES,ME,MSAD 61,NaN,TRANSFORMATION,"1877 ROOSEVELT TRAIL\nNAPLES, ME 4055\n(43.996...",43.9965 -70.6573
348,GOVERNOR JAMES B LONGLEY ELEM SCH,LEWISTON,ME,LEWISTON SCHOOL DEPARTMENT,NaN,TURNAROUND,"145 BIRCH ST\nLEWISTON, ME 4240\n(44.0943, -70...",44.0943 -70.2075
350,LIVERMORE FALLS HIGH SCHOOL,LIVERMORE FALLS,ME,MSAD 36,NaN,TRANSFORMATION,"25 CEDAR ST\nLIVERMORE FALLS, ME 4254\n(44.469...",44.4697 -70.1824
351,SUMNER MEMORIAL HIGH SCHOOL,SULLIVAN,ME,FLANDERS BAY CSD,NaN,TRANSFORMATION,"2456 US HWY 1\nSULLIVAN, ME 4664\n(44.5202, -6...",44.5202 -68.1972
353,CARRABEC HIGH SCHOOL,NORTH ANSON,ME,MSAD 74,NaN,TRANSFORMATION,"160 NORTH MAIN ST\nNORTH ANSON, ME 4958\n(44.8...",44.8595 -69.8949


In [282]:
school_data[school_data.State == 'NC']

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
460,BRUNSWICK LEARNING CENTER,SOUTHPORT,NC,BRUNSWICK COUNTY SCHOOLS,665360.0,TRANSFORMATION,"705 NORTH LORD ST\nSOUTHPORT, NC 28461\n(33.92...",33.9253 -78.0227
454,FAIRMONT HIGH,FAIRMONT,NC,ROBESON COUNTY SCHOOLS,903372.0,TRANSFORMATION,"5419 OLD STAGE RD\nFAIRMONT, NC 28340\n(34.480...",34.4801 -79.0962
449,LUMBERTON SENIOR HIGH,LUMBERTON,NC,ROBESON COUNTY SCHOOLS,1726.0,TRANSFORMATION,"3901 FAYETTEVILLE RD\nLUMBERTON, NC 28358\n(34...",34.6455 -79.0001
467,ANSON CHALLENGE ACADEMY,WADESBORO,NC,ANSON COUNTY SCHOOLS,730689.0,RESTART,"HWY 742 SOUTH\nWADESBORO, NC 28170\n(34.9656, ...",34.9656 -80.0673
469,WALKER-SPIVEY,FAYETTEVILLE,NC,CUMBERLAND COUNTY SCHOOLS,630752.0,TRANSFORMATION,"500 FISHER ST\nFAYETTEVILLE, NC 28301\n(35.038...",35.0389 -78.8726
468,E E WADDELL HIGH,CHARLOTTE,NC,CHARLOTTE-MECKLENBURG SCHOOLS,693605.0,TRANSFORMATION,"7030 NATIONS FORD RD\nCHARLOTTE, NC 28217\n(35...",35.1553 -80.9002
451,WEST MECKLENBURG HIGH,CHARLOTTE,NC,CHARLOTTE-MECKLENBURG SCHOOLS,879391.0,TRANSFORMATION,"7400 TUCKASEEGEE RD\nCHARLOTTE, NC 28214\n(35....",35.2473 -80.9529
457,WARLICK SCHOOL,GASTONIA,NC,GASTON COUNTY SCHOOLS,766415.0,TURNAROUND,"1316 SPENCER MOUNTAIN RD\nGASTONIA, NC 28054\n...",35.2815 -81.1333
459,GOLDSBORO HIGH,GOLDSBORO,NC,WAYNE COUNTY PUBLIC SCHOOLS,962048.0,TRANSFORMATION,"901 EAST BEECH ST\nGOLDSBORO, NC 27530\n(35.38...",35.386 -77.9812
450,SOUTH CENTRAL HIGH,WINTERVILLE,NC,PITT COUNTY SCHOOLS,946400.0,TRANSFORMATION,"570 WEST FORLINES RD\nWINTERVILLE, NC 28590\n(...",35.5394 -77.4282


In [283]:
school_data[school_data.State == 'NM']

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
498,R. SARRACINO MIDDLE,SOCORRO,NM,SOCORRO CONSOLIDATED SCHOOLS,975000.0,TRANSFORMATION,"1425 EL CAMINO REAL\nSOCORRO, NM 87801\n(34.08...",34.0832 -106.899
504,ALAMO NAVAJO SCHOOL,MAGDALENA,NM,ALAMO NAVAJO SCHOOL,NaN,TRANSFORMATION,"PO BOX 907\nMAGDALENA, NM 87825\n(34.1167, -10...",34.1167 -107.244
500,EL CAMINO REAL ACADEMY,ALBUQUERQUE,NM,ALBUQUERQUE PUBLIC SCHOOLS,11.0,TRANSFORMATION,"3713 ISLETA BLVD SW\nALBUQUERQUE, NM 87105\n(3...",35.0225 -106.683
499,LAGUNA-ACOMA HIGH,NEW LAGUNA,NM,GRANTS-CIBOLA COUNTY SCHOOLS,15.0,TRANSFORMATION,"HWY 66\nNEW LAGUNA, NM 87038\n(35.0463, -107.413)",35.0463 -107.413
503,ERNIE PYLE MIDDLE,ALBUQUERQUE,NM,ALBUQUERQUE PUBLIC SCHOOLS,2.0,TRANSFORMATION,"1820 VALDORA DR SW\nALBUQUERQUE, NM 87105\n(35...",35.05 -106.681
501,T'SIYA ELEMENTARY AND MIDDLE SCHOOL,ZIA PUEBLO,NM,T'SIYA ELEMENTARY AND MIDDLE SCHOOL,NaN,TRANSFORMATION,"1000 BORREGO CANYON ROAD\nZIA PUEBLO, NM 87053...",35.507 -106.72
505,RAMIREZ THOMAS ELEM,SANTA FE,NM,SANTA FE PUBLIC SCHOOLS,125.0,TURNAROUND,"610 ALTA VISTA\nSANTA FE, NM 87505\n(35.6705, ...",35.6705 -105.953
507,DIBE YAZHI HABLTI'N O'LT'A INC,CROWNPOINT,NM,DIBE YAZHI HABLTI'N O'LT'A INC,NaN,TRANSFORMATION,"PO BOX 679\nCROWNPOINT, NM 87313\n(35.6781, -1...",35.6781 -108.151
502,CROWNPOINT HIGH,CROWNPOINT,NM,GALLUP-MCKINLEY CTY SCHOOLS,2.0,TRANSFORMATION,"1 CHACO DR\nCROWNPOINT, NM 87313\n(35.6963, -1...",35.6963 -108.152
508,LYBROOK ELEMENTARY,CUBA,NM,JEMEZ MOUNTAIN PUBLIC SCHOOLS,500000.0,TRANSFORMATION,"HCR 17 BOX 350\nCUBA, NM 87013\n(35.9969, -107...",35.9969 -107.297


In [284]:
school_data[school_data.State == 'SD']

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
687,FAMILY IMMERSION CENTER MIDDLE - 53,SIOUX FALLS,SD,SIOUX FALLS SCHOOL DISTRICT 49-5,168131.0,TURNAROUND,"201 NORTH WEST AVE\nSIOUX FALLS, SD 57104\n(43...",43.5486 -96.7468
688,NEW UNDERWOOD HIGH SCHOOL - 01,NEW UNDERWOOD,SD,NEW UNDERWOOD SCHOOL DISTRICT 51-3,362759.0,TRANSFORMATION,"300 EAST ASH\nNEW UNDERWOOD, SD 57761\n(44.094...",44.0942 -102.834
686,SITTING BULL SCHOOL,LITTLE EAGLE,SD,SITTING BULL SCHOOL,NaN,TRANSFORMATION,"1 SCHOOL STREET, PO BOX 26\nLITTLE EAGLE, SD 5...",45.6847 -100.801


In [285]:
school_data[school_data.State == 'TX']

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
708,SENTRY TECHNOLOGY PREP SCH,BROWNSVILLE,TX,ONE STOP MULTISERVICE CHARTER SCHOOL,NaN,TRANSFORMATION,"508 EAST ELIZABETH\nBROWNSVILLE, TX 78520\n(25...",25.9057 -97.5031
715,MERCEDES ACADEMIC ACADEMY,MERCEDES,TX,MERCEDES ISD,NaN,TRANSFORMATION,"333 SOUTH OHIO ST\nMERCEDES, TX 78570\n(26.148...",26.1486 -97.9122
712,MID-VALLEY ACADEMY,MERCEDES,TX,MID-VALLEY ACADEMY,NaN,TRANSFORMATION,"103 EAST 2ND ST\nMERCEDES, TX 78570\n(26.15, -...",26.15 -97.9095
720,ONE STOP MULTISERVICE,WESLACO,TX,ONE STOP MULTISERVICE CHARTER SCHOOL,NaN,TRANSFORMATION,"615 SOUTH INTERNATIONAL\nWESLACO, TX 78596\n(2...",26.1538 -97.9584
735,MID-VALLEY ACADEMY (9-12),SAN BENITO,TX,MID-VALLEY ACADEMY,NaN,TRANSFORMATION,"1785 WEST BUSINESS HWY 77\nSAN BENITO, TX 7858...",26.1558 -97.6512
748,MID-VALLEY ACADEMY-MCALLEN,MCALLEN,TX,MID-VALLEY ACADEMY,NaN,TRANSFORMATION,"200 NORTH 17TH ST\nMCALLEN, TX 78501\n(26.206,...",26.206 -98.2375
705,JUAREZ-LINCOLN H S,LA JOYA,TX,LA JOYA ISD,NaN,TRANSFORMATION,"801 NORTH COYOTE DR\nLA JOYA, TX 78560\n(26.23...",26.2358 -98.4771
741,LA JOYA PALMVIEW H S,LA JOYA,TX,LA JOYA ISD,NaN,TRANSFORMATION,"603 NORTH COYOTE DR\nLA JOYA, TX 78560\n(26.23...",26.2358 -98.4771
726,JUVENILE DETENTION CTR,EDINBURG,TX,EDINBURG CISD,NaN,TRANSFORMATION,"3100 SOUTH HWY 281\nEDINBURG, TX 78539\n(26.27...",26.2749 -98.1703
743,CHILDREN OF THE SUN,RIO GRANDE CITY,TX,ONE STOP MULTISERVICE CHARTER SCHOOL,NaN,TRANSFORMATION,"5324 EAST US HWY 83 STE 2\nRIO GRANDE CITY, TX...",26.3549 -98.7723


# Begin to plot the data to see how to assess the funding.

First do a histogram of the dollars by number of schools


In [290]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import plotly.plotly as py

funding_levels = school_data['Amount']
layout = go.Layout(title="Grant Funding 2010/11 Academic Year",
                xaxis=dict(title='$ Funding Level'),
                yaxis=dict(title='Number of Schools'))
data = [go.Histogram(x=funding_levels)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar')


# Look at the smaller dollar amounts of funding


In [297]:

bool_series = school_data['Amount'] <= 50000 
funding_levels = school_data[bool_series].Amount
layout = go.Layout(title="Grant Funding 2010/11 Academic Year <= $50,000",
                xaxis=dict(title='$ Funding Level'),
                yaxis=dict(title='Number of Schools'))
data = [go.Histogram(x=funding_levels)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar')


# Look at the number of grants funded by model selected


In [299]:
model = school_data.Model_Selected
layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Model",
                xaxis=dict(title='Model'),
                yaxis=dict(title='Number of Schools'))
data = [go.Histogram(x=model)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar')


# Look at the funding by school for CLOSURE


In [309]:
bool_series = school_data.Model_Selected == 'CLOSURE'
model = school_data[bool_series].Amount
school = school_data[bool_series].School_Name
layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Closure Model",
                xaxis=dict(title=''),
                yaxis=dict(title='Funding Amount'))
data = [go.Bar(x=school, y=model)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar')


# Look at the funding by school for RESTART

In [311]:
bool_series = school_data.Model_Selected == 'RESTART'
model = school_data[bool_series].Amount
school = school_data[bool_series].School_Name
layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Restart Model",
                xaxis=dict(title=''),
                yaxis=dict(title='Funding Amount'))
data = [go.Bar(x=school, y=model)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar')


# Next look at the TURNAROUND model.

Because there are so many schools, first just look at a histogram to see if the dollar is there.  This shows an interesting grouping for this type of model.  Most funding is in the 100-200K range.



In [323]:
bool_series = school_data.Model_Selected == 'TURNAROUND'
model = school_data[bool_series].Amount
school = school_data[bool_series].School_Name
layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Turnaround",
                xaxis=dict(title='Funding Amount'),
                yaxis=dict(title='Number of Schools'))
data = [go.Histogram(x=model)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar')




Narrow the range, to see if the dollar is there. 


In [324]:
bool_series = (school_data['Model_Selected'] == 'TURNAROUND') & (school_data['Amount'] <= 100000)
model = school_data[bool_series].Amount
school = school_data[bool_series].School_Name
layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Turnaround <= $100K Funding",
                xaxis=dict(title='School'),
                yaxis=dict(title='Funding Amount'))
data = [go.Bar(x=school, y=model)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar')


# Next look at the Transformation model.

There seems to be a group in the 0-50K range.  Narrow the range to less than 1000, to see if the dollar is there.  As it wasn't in the other models, it is expected to be here.  There are 29 schools with less than 1000.  Run a bar graph to see the schools and funding levels.  Eliminate all below 200 when computing statistics as these are most likely errors as that amount of money is very little to accomplish the objective of the model.


In [321]:
bool_series = school_data.Model_Selected == 'TRANSFORMATION'
model = school_data[bool_series].Amount
school = school_data[bool_series].School_Name
layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Transformation",
                xaxis=dict(title='Funding Amount'),
                yaxis=dict(title='Number of Schools'))
data = [go.Histogram(x=model)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar')


In [328]:
bool_series = (school_data['Model_Selected'] == 'TRANSFORMATION') & (school_data['Amount'] <= 1000)
model = school_data[bool_series].Amount
school = school_data[bool_series].School_Name
layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Transformation",
                xaxis=dict(title='Funding Amount'),
                yaxis=dict(title='Number of Schools'))
data = [go.Histogram(x=model)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar')


In [329]:
bool_series = (school_data['Model_Selected'] == 'TRANSFORMATION') & (school_data['Amount'] <= 1000)
model = school_data[bool_series].Amount
school = school_data[bool_series].School_Name
layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Transformation <= $1,000 Funding",
                xaxis=dict(title=''),
                yaxis=dict(title='Funding Amount'))
data = [go.Bar(x=school, y=model)]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='jupyter-basic_bar')


# Check to see how many unique records will be used for the analysis

Eliminate records where funding was less than 200 or not provided.  Expect 724 records

In [350]:
bool_series = school_data.Amount >= 200
recordcount1 = school_data[bool_series]
recordcount1


,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
192,HOMESTEAD SENIOR HIGH SCHOOL,HOMESTEAD,FL,DADE,845263.0,TRANSFORMATION,"2351 SE 12TH AVE\nHOMESTEAD, FL 33034\n(25.450...",25.4507 -80.4589
214,PINE VILLA ELEMENTARY SCHOOL,MIAMI,FL,DADE,267620.0,TRANSFORMATION,"21799 SW 117TH COUROUTE\nMIAMI, FL 33170\n(25....",25.5644 -80.3814
183,MIAMI SOUTHRIDGE SENIOR HIGH,MIAMI,FL,DADE,115447.0,TURNAROUND,"19355 SW 114TH AVE\nMIAMI, FL 33157\n(25.5862,...",25.5862 -80.3751
237,FREDERICK R. DOUGLASS ELEMENTARY,MIAMI,FL,DADE,344819.0,TURNAROUND,"314 NW 12TH ST\nMIAMI, FL 33136\n(25.7854, -80...",25.7854 -80.2007
235,BOOKER T. WASHINGTON SENIOR HIGH,MIAMI,FL,DADE,644329.0,TURNAROUND,"1200 NW 6TH AVE\nMIAMI, FL 33136\n(25.7865, -8...",25.7865 -80.2048
187,MIAMI JACKSON SENIOR HIGH SCHOOL,MIAMI,FL,DADE,816787.0,TRANSFORMATION,"1751 NW 36TH ST\nMIAMI, FL 33142\n(25.8104, -8...",25.8104 -80.2251
248,CHARLES R. DREW MIDDLE SCHOOL,MIAMI,FL,DADE,369914.0,TURNAROUND,"1801 NW 60TH ST\nMIAMI, FL 33142\n(25.8298, -8...",25.8298 -80.2289
229,MIAMI EDISON MIDDLE SCHOOL,MIAMI,FL,DADE,407497.0,TURNAROUND,"6101 NW 2ND AVE\nMIAMI, FL 33127\n(25.8317, -8...",25.8317 -80.1998
205,MIAMI EDISON SENIOR HIGH SCHOOL,MIAMI,FL,DADE,618351.0,TURNAROUND,"6161 NW 5TH COUROUTE\nMIAMI, FL 33127\n(25.832...",25.8322 -80.2054
218,HOLMES ELEMENTARY SCHOOL,MIAMI,FL,DADE,276644.0,TRANSFORMATION,"1175 NW 67 ST\nMIAMI, FL 33150\n(25.8358, -80....",25.8358 -80.2163


In [351]:
recordcount1.describe()


,Amount
count,724.000000
mean,440008.017956
std,279200.953793
min,1036.000000
25%,175469.750000
50%,430568.500000
75%,686660.250000
max,997852.000000


In [352]:
recordcount1.describe(include=['object'])


,School_Name,City,State,District_Name,Model_Selected,Location,GPS
count,724,724,724,724,724,724,724
unique,717,366,46,355,4,716,715
top,EAST HIGH SCHOOL,PHILADELPHIA,CA,PHILADELPHIA CITY SD,TRANSFORMATION,"6941 NORTH CENTRAL ST\nPORTLAND, OR 97203\n(45...",45.5899 -122.739
freq,2,27,87,25,514,3,3


Display 107 records being eliminated.  See that there are 107 - 74 with no amount and 33 with an amount which is less than 200

In [345]:
recordcount = school_data[~bool_series]
pd.set_option('display.max_rows', 110)
recordcount.count()


School_Name       107
City              107
State             107
District_Name     107
Amount             33
Model_Selected    101
Location          107
GPS               107
dtype: int64

In [346]:
recordcount

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS
708,SENTRY TECHNOLOGY PREP SCH,BROWNSVILLE,TX,ONE STOP MULTISERVICE CHARTER SCHOOL,NaN,TRANSFORMATION,"508 EAST ELIZABETH\nBROWNSVILLE, TX 78520\n(25...",25.9057 -97.5031
715,MERCEDES ACADEMIC ACADEMY,MERCEDES,TX,MERCEDES ISD,NaN,TRANSFORMATION,"333 SOUTH OHIO ST\nMERCEDES, TX 78570\n(26.148...",26.1486 -97.9122
712,MID-VALLEY ACADEMY,MERCEDES,TX,MID-VALLEY ACADEMY,NaN,TRANSFORMATION,"103 EAST 2ND ST\nMERCEDES, TX 78570\n(26.15, -...",26.15 -97.9095
720,ONE STOP MULTISERVICE,WESLACO,TX,ONE STOP MULTISERVICE CHARTER SCHOOL,NaN,TRANSFORMATION,"615 SOUTH INTERNATIONAL\nWESLACO, TX 78596\n(2...",26.1538 -97.9584
735,MID-VALLEY ACADEMY (9-12),SAN BENITO,TX,MID-VALLEY ACADEMY,NaN,TRANSFORMATION,"1785 WEST BUSINESS HWY 77\nSAN BENITO, TX 7858...",26.1558 -97.6512
748,MID-VALLEY ACADEMY-MCALLEN,MCALLEN,TX,MID-VALLEY ACADEMY,NaN,TRANSFORMATION,"200 NORTH 17TH ST\nMCALLEN, TX 78501\n(26.206,...",26.206 -98.2375
705,JUAREZ-LINCOLN H S,LA JOYA,TX,LA JOYA ISD,NaN,TRANSFORMATION,"801 NORTH COYOTE DR\nLA JOYA, TX 78560\n(26.23...",26.2358 -98.4771
741,LA JOYA PALMVIEW H S,LA JOYA,TX,LA JOYA ISD,NaN,TRANSFORMATION,"603 NORTH COYOTE DR\nLA JOYA, TX 78560\n(26.23...",26.2358 -98.4771
726,JUVENILE DETENTION CTR,EDINBURG,TX,EDINBURG CISD,NaN,TRANSFORMATION,"3100 SOUTH HWY 281\nEDINBURG, TX 78539\n(26.27...",26.2749 -98.1703
743,CHILDREN OF THE SUN,RIO GRANDE CITY,TX,ONE STOP MULTISERVICE CHARTER SCHOOL,NaN,TRANSFORMATION,"5324 EAST US HWY 83 STE 2\nRIO GRANDE CITY, TX...",26.3549 -98.7723


In [348]:
recordcount.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 708 to 287
Data columns (total 8 columns):
School_Name       107 non-null object
City              107 non-null object
State             107 non-null object
District_Name     107 non-null object
Amount            33 non-null float64
Model_Selected    101 non-null object
Location          107 non-null object
GPS               107 non-null object
dtypes: float64(1), object(7)
memory usage: 7.5+ KB


# Analyze the grant funding

First make a dataframe with just the records with valid awards - add the latitude and longitude columns for using with maps.

In [364]:
gps1 = recordcount1.GPS.str.split(" ").apply(pd.Series)
gps1.columns = ('Latitude', 'Longitude')
#gps1
grantawards=recordcount1.join(gps1, how = 'outer')
grantawards

,School_Name,City,State,District_Name,Amount,Model_Selected,Location,GPS,Latitude,Longitude
192,HOMESTEAD SENIOR HIGH SCHOOL,HOMESTEAD,FL,DADE,845263.0,TRANSFORMATION,"2351 SE 12TH AVE\nHOMESTEAD, FL 33034\n(25.450...",25.4507 -80.4589,25.4507,-80.4589
214,PINE VILLA ELEMENTARY SCHOOL,MIAMI,FL,DADE,267620.0,TRANSFORMATION,"21799 SW 117TH COUROUTE\nMIAMI, FL 33170\n(25....",25.5644 -80.3814,25.5644,-80.3814
183,MIAMI SOUTHRIDGE SENIOR HIGH,MIAMI,FL,DADE,115447.0,TURNAROUND,"19355 SW 114TH AVE\nMIAMI, FL 33157\n(25.5862,...",25.5862 -80.3751,25.5862,-80.3751
237,FREDERICK R. DOUGLASS ELEMENTARY,MIAMI,FL,DADE,344819.0,TURNAROUND,"314 NW 12TH ST\nMIAMI, FL 33136\n(25.7854, -80...",25.7854 -80.2007,25.7854,-80.2007
235,BOOKER T. WASHINGTON SENIOR HIGH,MIAMI,FL,DADE,644329.0,TURNAROUND,"1200 NW 6TH AVE\nMIAMI, FL 33136\n(25.7865, -8...",25.7865 -80.2048,25.7865,-80.2048
187,MIAMI JACKSON SENIOR HIGH SCHOOL,MIAMI,FL,DADE,816787.0,TRANSFORMATION,"1751 NW 36TH ST\nMIAMI, FL 33142\n(25.8104, -8...",25.8104 -80.2251,25.8104,-80.2251
248,CHARLES R. DREW MIDDLE SCHOOL,MIAMI,FL,DADE,369914.0,TURNAROUND,"1801 NW 60TH ST\nMIAMI, FL 33142\n(25.8298, -8...",25.8298 -80.2289,25.8298,-80.2289
229,MIAMI EDISON MIDDLE SCHOOL,MIAMI,FL,DADE,407497.0,TURNAROUND,"6101 NW 2ND AVE\nMIAMI, FL 33127\n(25.8317, -8...",25.8317 -80.1998,25.8317,-80.1998
205,MIAMI EDISON SENIOR HIGH SCHOOL,MIAMI,FL,DADE,618351.0,TURNAROUND,"6161 NW 5TH COUROUTE\nMIAMI, FL 33127\n(25.832...",25.8322 -80.2054,25.8322,-80.2054
218,HOLMES ELEMENTARY SCHOOL,MIAMI,FL,DADE,276644.0,TRANSFORMATION,"1175 NW 67 ST\nMIAMI, FL 33150\n(25.8358, -80....",25.8358 -80.2163,25.8358,-80.2163


# Plot

Too many schools to plot the types of grants and amounts funded by school and type of model.  Do a histogram for each funding Model.


In [414]:
bool_series1 = grantawards['Model_Selected'] == 'TRANSFORMATION'
model1 = grantawards[bool_series1].Amount
longitude1 = grantawards[bool_series1].Longitude
latitude1 = grantawards[bool_series1].Latitude

bool_series2 = grantawards['Model_Selected'] == 'RESTART'
model2 = grantawards[bool_series2].Amount
longitude2 = grantawards[bool_series2].Longitude
latitude2 = grantawards[bool_series2].Latitude

bool_series3 = grantawards['Model_Selected'] == 'CLOSURE'
model3 = grantawards[bool_series3].Amount
longitude3 = grantawards[bool_series3].Longitude
latitude3 = grantawards[bool_series3].Latitude

bool_series4 = grantawards['Model_Selected'] == 'TURNAROUND'
model4 = grantawards[bool_series4].Amount
longitude4 = grantawards[bool_series4].Longitude
latitude4 = grantawards[bool_series4].Latitude

trace_transformation = go.Histogram(x=model1,
                  name='Transformation',
                  marker=dict(color='#ffcdd2'))

trace_restart = go.Histogram(x=model2,
                name='Restart',
                marker=dict(color='#A2D5F2'))

trace_closure = go.Histogram(x=model3,
                name='Closure',
                marker=dict(color='#59606D'))

trace_turnaround = go.Histogram(x=model4,
                name='Turnaround',
                marker=dict(color='#5965F2'))

data = [trace_transformation, trace_restart, trace_closure, trace_turnaround]

layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Model",
                xaxis=dict(title='Funding Amount'),
                yaxis=dict(title='Number of Schools'))

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='jupyter-styled_bar')

# Compute statistics for each type of funding

Not sure why, but the numbers vary slightly in the statistics computed by describe() vs the graph statistics.  Could be rounding precision or population vs sample calculations.


In [392]:
stats = pd.DataFrame(group.describe().rename(columns={'Amount':name}).squeeze()
                         for name, group in grantawards.groupby('Model_Selected'))

stats

,count,mean,std,min,25%,50%,75%,max
CLOSURE,16.0,75155.500000,93610.918355,5000.0,11546.75,47958.5,61330.0,300000.0
RESTART,33.0,503412.424242,294183.676162,17529.0,189201.00,517340.0,730689.0,980283.0
TRANSFORMATION,514.0,458607.474708,276267.922927,1036.0,187874.50,461206.5,699599.0,997852.0
TURNAROUND,161.0,403891.086957,269892.652693,11651.0,162698.00,363928.0,633527.0,966394.0


In [408]:
trace1 = go.Box(
 y = model1,
 name = 'Transformation',
 marker=dict(color='#ffcdd2'))
trace2 = go.Box(
 y = model2,
 name = 'Restart',
 marker=dict(color='#A2D5F2'))
trace3 = go.Box(
 y = model3,
 name = 'Closure',
 marker=dict(color='#59606D'))
trace4 = go.Box(
 y = model4,
 name = 'Turnaround',
 marker=dict(color='#5965F2'))
    
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Model",
                xaxis=dict(title='Funding Model'),
                yaxis=dict(title='Amount Funded'))
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='jupyter-styled_box')


# Look at the geographic distribution in the US.  

Additionally, this confirms that HI is not included.  A test for that wasn't run earlier, but one state was missing from the original list of 50 (vs 51 because of DC).

In [419]:
trace1 = dict(
  type='scattergeo',
  lon = longitude1, lat = latitude1,
  name = 'Transformation',
  marker=dict(color='#ffcdd2'))
trace2 = dict(
  type='scattergeo',
  lon = longitude2, lat = latitude2,
  name = 'Restart',
  marker=dict(color='#A2D5F2'))
trace3 = dict(
  type='scattergeo',
  lon = longitude3, lat = latitude3,
  name = 'Closure',
  marker=dict(color='#59606D'))
trace4 = dict(
  type='scattergeo',
  lon = longitude4, lat = latitude4,
  name = 'Turnaround',
  marker=dict(color='#5965F2'))

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(title="Grant Funding 2010/11 Academic Year by Model")
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='jupyter-styled_geoscatter')

This graph is more detailed in terms of the locaions in the United States.

In [434]:
mapbox_access_token = 'pk.eyJ1Ijoic2NhdGFub3NvIiwiYSI6ImNqbzJpMWtwZDBtMjQza3FxaHRjaHp2emsifQ.qPYije503m1QBkCFusgNRA'

#site_lat = latitude1
#site_lon = longitude1
#locations_name = grantawards.School_Name
#       marker=dict(
  #          size=17,
   #         color='rgb(255, 0, 0)',
    #        opacity=0.7
     #   ),
#        marker=dict(
 #           size=8,
  #          color='rgb(242, 177, 172)',
   #         opacity=0.7
    #    ),       
        
data = [
    go.Scattermapbox(
        lat=latitude1,
        lon=longitude1,
        mode='markers',
        name = 'Transformation',
        marker=dict(color='#ffcdd2'),
        text=grantawards.School_Name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=latitude2,
        lon=longitude2,
        mode='markers',
        name = 'Restart',
        marker=dict(color='#A2D5F2'),
        text=grantawards.School_Name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=latitude3,
        lon=longitude3,
        mode='markers',
        name = 'Closure',
        marker=dict(color='#59606D'),
        text=grantawards.School_Name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=latitude4,
        lon=longitude4,
        mode='markers',
        name = 'Turnaround',
        marker=dict(color='#5965F2'),
        text=grantawards.School_Name,
        hoverinfo='text'
    )]


layout = go.Layout(
    title='Grant Funding 2010/11 Academic Year',
    autosize=True,
    hovermode='closest',
    showlegend=True,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38,
            lon=-94
        ),
        pitch=0,
        zoom=3,
        style='light'
    ),
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='jupyter-Grant Funding')